# Librerias a usar para el parser

In [38]:
from typing import Union, Optional, List, Dict, Iterator, Tuple, Literal
from dataclasses import dataclass
from pathlib import Path
from datetime import date
import pandas as pd
import re

# Mejoras de constantes

In [39]:
# Constantes para nombres de grupos de la regex
G_EST      = 'est'
G_VAL_571  = 'val_571'
G_VAL_572  = 'val_572'
G_VAL_573  = 'val_573'
G_VAL_574  = 'val_574'
G_VAL_83   = 'val_83'

SCALE: Dict[str, float] = {'6': 1e4, '8': 1e5, '0': 1e3}

@dataclass
class Medida:
    orientation: Literal["ini", "fin"]
    dif_dist: Optional[float] = None
    total_dist: Optional[float] = None
    dif_height: Optional[float] = None
    med_ini: Optional[float] = None
    # metadatos útiles para trazabilidad
    est: Optional[str] = None
    file: Optional[str] = None

# Paths y REGEX

In [36]:
# Paths y expresiones

PATH = Path("G:\\Otros ordenadores\\Mi PC\\IGAC\\archivosPrueba\\Archivos_Pruebas\\L5_GSI_1")
L5Crudos = PATH / 'CRUDOS_L5'
L5Orden = PATH / 'Orden_L5' / 'L5_Principio_Linea;Fin_linea.txt'
crudos = [p for p in L5Crudos.iterdir() if p.is_file() is True]

# BLOCK: Patron que separa los bloques
BLOCK_RE = re.compile(r'^\s*(?P<grupo>[^+\s]+)\+[^?]*\?.*?(?P<codigo>[1-4]|10)\b')

# VERT: Patron que estandariza los vertices XXXX-XXXX-09
VERT_RE = re.compile(r'^([A-Z0-9]+)([A-Z]{2,4})(\d{1,2})$')

# FECHA6: Extrae fechas
FECHA6_RE = re.compile(r'(\d{6})')

# INI: siguiente línea del bloque. Se exige 83..[068]<valor> y se toma como ini si <valor> == 0
INI_RE = re.compile(r"""
    (?P<est>[A-Z0-9]+)\s+
    (?P<idx_83>83)\.\.(?P<p5_83>[25])(?P<p6_83>[068])(?P<val_83>[+-]?\d+)
""", re.X)

# FIN: línea BF completa (573, 574, 83)
FIN_RE_BF = re.compile(r"""
    (?P<est>[A-Z0-9]+)\s+
    (?P<idx_573>573)\.\.(?P<p6_573>[068])(?P<val_573>[+-]?\d+)\s+
    (?P<idx_574>574)\.\.(?P<p6_574>[068])(?P<val_574>[+-]?\d+)\s+
    (?P<idx_83>83)\.\.(?P<p5_83>[0125])(?P<p6_83>[068])(?P<val_83>[+-]?\d+)
""", re.X)

# Estandarizador de Nomenclaturas

In [5]:
# Función que estandariza las nomenclaturas
def vert_std(codigo:str) -> str:
    code_mayusc = codigo.replace("-", "").replace(" ", "").upper()
    m = VERT_RE.match(code_mayusc)
    if m:
        g1, g2, g3 = m.groups()
        return f"{g1}-{g2}-{g3}"
    
    return codigo

print(vert_std('A50BE2'))

A50-BE-2


# Clasificador de Orden

In [8]:
# Función que ordena el archivo orden
def clasi_orden(forden:Path) -> List[List[Dict[str,str]]]:
    ida: List[Dict[str, str]] = []
    reg: List[Dict[str, str]] = []

    with open(forden, "r", encoding="utf-8") as f:
        # suponemos primera línea encabezado
        for i, linea in enumerate(f.readlines()[1:]):
            linea = linea.strip()
            if not linea or ";" not in linea:
                continue
            a, b = linea.split(";")
            a = vert_std(a)
            b = vert_std(b)
            if i % 2 == 0:
                ida.append({"ini" : a, "fin": b})
            else:
                reg.append({"ini" : a, "fin": b})

    return [ida,reg]

print(clasi_orden(L5Orden))

[[{'ini': 'B70-CW-7', 'fin': '23660003'}, {'ini': '23660003', 'fin': 'A2-CB-3'}, {'ini': 'A2-CB-3', 'fin': 'A3-CB-3'}, {'ini': 'A3-CB-3', 'fin': '4-CB-3'}, {'ini': '4-CB-3', 'fin': '5-CB-3'}, {'ini': '5-CB-3', 'fin': '6-CB-3'}, {'ini': '6-CB-3', 'fin': '23189001'}, {'ini': '23189001', 'fin': '8-CB-3'}, {'ini': '8-CB-3', 'fin': '9-CB-3'}, {'ini': '9-CB-3', 'fin': '10-CB-3'}, {'ini': '10-CB-3', 'fin': '23189002'}, {'ini': '23189002', 'fin': '12-CB-3'}, {'ini': '12-CB-3', 'fin': '13-CB-3'}, {'ini': '13-CB-3', 'fin': '23189003'}, {'ini': '23189003', 'fin': 'A16-CB-3'}, {'ini': 'A16-CB-3', 'fin': '23189004'}, {'ini': '23189004', 'fin': '18-CB-3'}, {'ini': '18-CB-3', 'fin': '23189005'}, {'ini': '23189005', 'fin': '19-CB-3'}, {'ini': '19-CB-3', 'fin': '15-TE-2'}, {'ini': '15-TE-2', 'fin': 'A20-CB-3'}, {'ini': 'A20-CB-3', 'fin': 'A21-CB-3'}, {'ini': 'A21-CB-3', 'fin': 'A22-CB-3'}, {'ini': 'A22-CB-3', 'fin': '23-CB-3'}, {'ini': '23-CB-3', 'fin': '23189006'}, {'ini': '23189006', 'fin': 'A25-CB-3

# Generador de Fechas

In [10]:
def ext_fecha(fname:str):
    m = FECHA6_RE.search(fname)
    if not m:
        return None
    
    y, mth, d = int(m.group(1)[:2]), int(m.group(1)[2:4]), int(m.group(1)[4:6])

    # regla para siglo: 00–49 → 2000s, 50–99 → 1900s
    year = 2000 + y if y < 50 else 1900 + y

    try:
        return date(year, mth, d).isoformat()
    except ValueError:
        return None
    
print(ext_fecha('100517'))

2010-05-17


# Parser de estaciones de Inicio

In [43]:
archivos = sorted([p for p in L5Crudos.iterdir() if p.is_file() is True])
archivo_prueba = archivos[10]

def parser_med(file:Path, expr:re.Match, dig5:str, dig6:str, ori:Literal['ini','fin']) -> Medida:
    # Verificaciones 
    if not file:
        raise ValueError("No hay un archivo asociado.")
    if not expr:
        raise ValueError(f"No hay expresión regular en {file.name}")
    if not dig5:
        est_txt = vert_std(expr.group(G_EST).lstrip('0')) if expr.groupdict().get(G_EST) else "¿est?"
        raise ValueError(f"No hay dígito para curvatura en el vértice {est_txt} en {file.name}")
    if ori not in ("ini", "fin"):
        raise ValueError(f"Orientación inválida '{ori}' en {file.name}")

    base = SCALE.get(dig6)
    if not base:
        raise RuntimeError(f"Digito no contemplado para {vert_std(expr.group(G_EST).lstrip('0'))} en {file.name}")

    try:
        est_txt = vert_std(expr.group(G_EST).lstrip('0'))
    except Exception:
        raise RuntimeError(f"Error desconocido para {vert_std(expr.group(G_EST).lstrip('0'))} en {file.name}")

    def _f(grp: str) -> float:
        try:
            return float(expr.group(grp)) / base
        except Exception as e:
            raise ValueError(f"Grupo '{grp}' inválido o ausente en {file.name}") from e
        
    if dig5 == '2' and ori == 'fin':
        return Medida(orientation="fin", dif_dist=_f(G_VAL_573), total_dist=_f(G_VAL_574),
            dif_height=_f(G_VAL_83), est=est_txt, file=file.name)
    elif dig5 in ('2', '5') and ori == 'ini':
        return Medida(orientation="ini", med_ini=_f(G_VAL_83), est=est_txt, file=file.name)
    else:
        raise ValueError(
            f"Combinación no implementada: dig5='{dig5}', ori='{ori}' "
            f"para vértice {est_txt or '¿est?'} en {file.name}"
        )


def parser_ini(file: Path, curr_line: str, next_line: str) -> Optional[Medida]:
    if not file:
        raise ValueError("No hay un archivo asociado.")
    if not curr_line:
        raise RuntimeError(f"No se está almacenando línea actual en {file.name}")
    if not next_line:
        raise RuntimeError(f"No se está almacenando línea siguiente en {file.name}")

    # 1) Verifica que curr_line sea un bloque válido
    if not BLOCK_RE.search(curr_line.strip()):
        return None

    # 2) Extrae la línea 'ini'
    m_ini = INI_RE.search(next_line.strip())
    if not m_ini:
        return None

    d = m_ini.groupdict()
    dig5 = d.get("p5_83")
    dig6 = d.get("p6_83")
    if not dig5 or not dig6:
        est_txt = d.get("est", "¿est?")
        raise ValueError(f"Faltan p5_83/p6_83 para el vértice {est_txt} en {file.name}")

    # 3) Delega el cálculo al parser_med (retorna Medida o None)
    medida = parser_med(file=file, expr=m_ini, dig5=dig5, dig6=dig6, ori="ini")

    # 4) Si necesitas anexar info del bloque (fecha, id, etc.), puedes actualizar aquí:
    # if medida is not None:
    #     medida.algo_del_bloque = <extraído de m_block>

    return medida
    

with open(archivo_prueba,"r") as f:
    lineas = f.readlines()

for i, ln in enumerate(lineas):
    next_line = lineas[i + 1] if i + 1 < len(lineas) else None
    prev_line = lineas[i - 1] if i - 1 >= 0 else None
    if not next_line:
        continue

# Manejo de clases

In [128]:
linea = "110042+23001003 573..6+00005738 574..6+16214182 83..26-00002484"
linea2 = "110009+00000001 573..6-00010980 574..6+00321848 83..26+00012290"
linea3 = "110083+00A42CB3 573..6-00047557 574..6+17188834 83..26+00022618"
linea4 = "110121+00000012 573..6-00020700 574..6+15287876 83..26+00008417"
linea5 = "110056+50D-CU-1 571.28+00000006 572.28-00000052 573..8-00090868 574..8+84462336 83..28+00589637"

RE_PRUEBA = re.compile(r'''
^
(\d{6})\+                                          # 1: seis dígitos y '+'

(?P<id_vert>                                       # 2: token después del '+'
    (?:                                            #    alternancia excluyente
        (?=\d{8}(?=\s|$))(?!\d{0,4}0000)\d{8}      #    numérico: 8 dígitos y sin '0000'
      | (?=[A-Za-z0-9-]{3,8}(?=\s|$))               #    alfanumérico: 3–8, termina en espacio/fin
        (?=[A-Za-z0-9-]*[A-Za-z])                   #    contiene ≥1 letra
        [A-Za-z0-9-]{3,8}
    )
)
.*?                                                # separador
# ---- Bloque 573 (exactamente 15 caracteres) ----
(?P<b573>573\.\.[068](?P<med_573>[+-]\d{8}))\s+

# ---- Bloque 574 (exactamente 15 caracteres) ----
(?P<b574>574\.\.[068](?P<med_574>[+-]\d{8}))\s+

# ---- Bloque 83 (exactamente 15 caracteres) ----
(?P<b83>83\.\.(?P<corr_83>\d)?(?P<esc_83>\d)(?P<med_83>[+-]\d{8}))
''', re.VERBOSE)

ver = RE_PRUEBA.search(linea3)
if ver is not None:
    print(ver.groupdict())
else:
    print(None)


{'id_vert': '00A42CB3', 'b573': '573..6-00047557', 'med_573': '-00047557', 'b574': '574..6+17188834', 'med_574': '+17188834', 'b83': '83..26+00022618', 'corr_83': '2', 'esc_83': '6', 'med_83': '+00022618'}
